<!-- PROFILE LINK -->

<h1 style = "text-align:center; font-size: 30px">PROGRAMACIÓN CONCURRENTE Y DISTRIBUIDA</h1>
<a href ="https://github.com/samuu159/PCD">
<img src="https://avatars.githubusercontent.com/u/100205384?v=4" style="float:left;vertical-align:centre" width="110" height="110" title = "Christian Vlaldimir Sucuzhanay Arevalo 21535220">
<a href ="https://github.com/samuu159/PCD">
<img src="https://media-exp1.licdn.com/dms/image/C4D0BAQEu_Aa76fcmPw/company-logo_200_200/0/1630998679429?e=2159024400&v=beta&t=2feC9GG7RWCWizTqsK5HzDWB1TL2DOQuAVwnMf6FFvk" title = "M41" alt="UE" width="110" height="110" style="float:right;vertical-align:centre">

## CLASE CLIENTE

In [1]:
#Empezamos importando las bibliotecas necesarias
import threading
import sys
import socket
import pickle
import os

class Cliente():
	
	def __init__(self, name=input("Intoduzca el nickname:  "), host=input("Intoduzca la IP del servidor ?  "), port=int(input("Intoduzca el PUERTO del servidor ?  "))): #Función que pide al cliente el nickname(correo de la universidad) y el puerto
		self.s = socket.socket()
		self.s.connect((host, int(port)))#Conecta el host con el puerto

		self.name=name
		self.s.send(name.encode())#Pasamos el nickname al servidor para que tenga el dato y poder manejarlo después
		print('\n\tProceso con PID = ',os.getpid(), '\n\tHilo PRINCIPAL con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tTotal Hilos activos en este punto del programa =', threading.active_count())#Imprime los hilos que se van a utilizar
		threading.Thread(target=self.recibir, daemon=True).start()

		while True:
			msg = input('\n Nickname: ' + self.name + '\tEscriba texto ?   ** Enviar = ENTER   ** Salir Chat = 1 \n')#Menú que le va a salir al cliente en el cual va a aprecer el nickname y una opción para poder enviar un mensaje. Para salir del menú tendra que pulsar 1.
			if msg != '1' : self.enviar(msg)#Si el ususario escribe otra cosa que no es 1 envía el mensaje
			else:
				print(" **** Me piro vampiro; cierro socket y mato al CLIENTE con PID = ", os.getpid())#Mensaje que le va a salir cuando pulse 1 y cierre el cliente
				self.s.close()
				sys.exit()

	def recibir(self):#Función para poder ver los hilos que van a salir y los que están activos en este momento
		print('\nHilo RECIBIR con ID =',threading.currentThread().getName(), '\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count())
		while True:
			try:
				data = self.s.recv(32)
				if data: print(pickle.loads(data))
			except: pass

	def enviar(self, msg):#Función que sirve para que aparezca el nickname al enviar el mensaje
		mensaje = f"{self.name}: {msg}"
		self.s.send(pickle.dumps(mensaje))

arrancar = Cliente()#Función para arrancar el cliente

Intoduzca el nickname:   
Intoduzca la IP del servidor ?   
Intoduzca el PUERTO del servidor ?   


ValueError: invalid literal for int() with base 10: ''

## CLASE SERVIDOR

In [ ]:
#Empezamos importando las bibliotecas necesarias
import socket
import threading
import sys
import pickle
import os


class Servidor():#Creamos la clase servidor
	
	def __init__(self, host=socket.gethostname(), port=int(input("Que puerto quiere usar ? "))):#Función que va a pedir al usuario el puerto que quiere usar
		self.clientes = []#Creamos array de clientes
		self.name =[]#Creamos array de nombres
		print('\nSu IP actual es : ',socket.gethostbyname(host))#Muestra la IP actual 
		print('\n\tProceso con PID = ',os.getpid(), '\n\tHilo PRINCIPAL con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(), '\n\tTotal Hilos activos en este punto del programa =', threading.active_count())#Muestra los hilos totales del programa
		self.s = socket.socket()
		self.s.bind((str(host), int(port)))#Enlaza el host y el puerto
		self.s.listen(30)
		self.s.setblocking(False)
		

		threading.Thread(target=self.aceptarC, daemon=True).start()
		threading.Thread(target=self.procesarC, daemon=True).start()

		while True:
			msg = input('\n << SALIR = 1 >> \n')#Cerrar el servidor pulsando 1
			if msg == '1':
				print(" **** Me piro vampiro; cierro socket y mato SERVER con PID = ", os.getpid())#Mensaje que le va a salir al usuario cuando cierre el servidor
				self.s.close()
				sys.exit()
			else: pass

	def aceptarC(self):#Función que sirve para aceptar el mensaje enviado por el cliente
		print('\nHilo ACEPTAR con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count())#Muestra los hilos totales
		
		while True:
			try:
				conn, addr = self.s.accept()
				print(f"\nConexion aceptada via {addr}\n")#Conexion aceptada
				conn.setblocking(False)
				self.clientes.append(conn)
				data = conn.recv(32).decode()
				if data:#Creamos un if para ver si el nickname es aceptado o no por el servidor
					name = data
					print("Nickname aceptado" , name)
					self.name.append(name)
			except: pass

	def procesarC(self):#Creamos la función procesar los mensajes que va recibiendo del cliente
		print('\nHilo PROCESAR con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count())#Ver hilos totales
		while True:
			if len(self.clientes) > 0:
				for c in self.clientes:
					try:
						data = c.recv(32)
						if data: self.broadcast(data,c)
						with open('ue22166872AI1.txt', 'a') as f:#Creamos el txt para ir añadiendo todos los mensajes que vaya recibiendo el servidor
							f.write(pickle.loads(data) + '\n')#Los va poniendo uno detrás de otro
					except: pass

	def broadcast(self, msg, cliente):#Función para ver los clientes conectados e imprimir sus nicknames
		for c in self.clientes:
			print("Clientes conectados Right now = ", len(self.clientes), ' ',self.name, '\tMensaje de: ', pickle.loads(msg))#Imprime el nickname del cliente que ha escrito el mensaje
			try:
				if c != cliente: 
					
					c.send(msg)
			except: self.clientes.remove(c)

arrancar = Servidor() #Función para arrancar el servidor